In [1]:
import pandas as pd
import numpy as np
import os

# dataset_name = 'prepared_facebook_data'
# folder_name = 'facebook_comm_vol'
# dataset_name ='house_prices'
# folder_name = 'boston_house_prices'
# dataset_name = 'energy_consumption'
# folder_name = 'energy_consumption'
dataset_name = 'physics'
folder_name = 'physics'

DIR_WITH_FIRST_GROUP_FILES = f"../../other_data_experiments/{folder_name}/third experiment/groups/"
# res_BO_group1_prepared_facebook_data_1.csv
# res_BO_group2_prepared_facebook_data_1.csv

DIR_WITH_NEW_GROUP_FILES =  f"../../other_data_experiments/{folder_name}/fourth experiment/raw/"
# GRBO_S_2_group_with_100_1_prepared_facebook_data.csv
# GRBO_S_3_group_with_100_1_prepared_facebook_data.csv

DIR_TO_SAVE_RES = f"../../other_data_experiments/{folder_name}/fourth experiment/prepared/"

In [2]:
def group_best_params_with_stop(df, stop_iter_num, left_iter_num, right_iter_num):
    best_params = {}
    stop_iter_nums = {}
    best_score = {}

    tmp_df = df.loc[(df['iter_num'] >= left_iter_num) & (df['iter_num'] <= right_iter_num)].copy() #take group
    for i in tmp_df.run_number.unique(): 
      tmp_df_i = tmp_df.loc[tmp_df.run_number==i] #take run
      #cut iterations:
      tmp_df_i_copy = tmp_df_i.copy()
      tmp_df_i_copy.loc[:, "mean_test_score"] = tmp_df_i_copy.loc[
          :, "mean_test_score"].cummax()      

      try:
          curr_max=tmp_df_i_copy.groupby("mean_test_score").count().reset_index().sort_values(by = "mean_test_score")
          max_score = curr_max.loc[curr_max["mean_test_score"]>=stop_iter_num].index[0] #get first interval with iter_amount more than stop_iter_num
      except:
          max_score = tmp_df_i_copy.mean_test_score.max()

      tmp_stop = tmp_df_i.loc[tmp_df_i["mean_test_score"]==max_score].sort_values(by="iter_num").iloc[0]#get first element with best_score
    
      best_params[i] = tmp_stop['params']
      #best_score[i] = tmp_stop['mean_test_score']

      last_iter_num = tmp_stop['iter_num'] + stop_iter_num - 1
      if last_iter_num > right_iter_num:
          last_iter_num = right_iter_num

      stop_iter_nums[i] = last_iter_num
    return best_params, stop_iter_nums #, best_score       
   

In [3]:

for i in range(1, 11):
    first_group_df = pd.read_csv(DIR_WITH_FIRST_GROUP_FILES + f"res_BO_group1_{dataset_name}_{i}.csv") 
    first_group_df['iter_num'] = range(1, first_group_df.shape[0]+1)
    first_group_df['run_number'] = i

    for stop_iter_num in [10]:
        # prepare first group:
        _, first_stop_iter_num = group_best_params_with_stop(
            first_group_df, stop_iter_num = stop_iter_num,
            left_iter_num=1, right_iter_num=243 # first group
        )
        first_stop_iter_num = first_stop_iter_num[i]
        first_group_df_cutted = first_group_df.loc[first_group_df.iter_num <= first_stop_iter_num]
        
        # prepare second group:
        if first_stop_iter_num==243:
            second_group_df = pd.read_csv(DIR_WITH_FIRST_GROUP_FILES + f"res_BO_group2_{dataset_name}_{i}.csv") 
        else:
#             dataset_name = 'boston'
            second_group_df = pd.read_csv(DIR_WITH_NEW_GROUP_FILES + \
                                          f"GRBO_S_2_group_with_{stop_iter_num}_{i}_{dataset_name}.csv")
            
        second_group_df['iter_num'] = range(1, second_group_df.shape[0]+1)
        second_group_df['run_number'] = i
        _, second_stop_iter_num = group_best_params_with_stop(
            second_group_df, stop_iter_num = stop_iter_num,
            left_iter_num=1, right_iter_num=second_group_df.shape[0]
        )
        second_stop_iter_num = second_stop_iter_num[i]
        second_group_df_cutted = second_group_df.loc[second_group_df.iter_num <= second_stop_iter_num]
        
        # prepare third group:
        try:
            if (first_stop_iter_num==243) and (second_stop_iter_num==243):
                third_group_df = pd.read_csv(DIR_WITH_FIRST_GROUP_FILES + f"res_BO_group3_{dataset_name}_{i}.csv")
            else:
#                 dataset_name = 'boston'
                third_group_df = pd.read_csv(DIR_WITH_NEW_GROUP_FILES + \
                                              f"GRBO_S_3_group_with_{stop_iter_num}_{i}_{dataset_name}.csv")
#             dataset_name ='house_prices'


            third_group_df['iter_num'] = range(1, third_group_df.shape[0]+1)
            third_group_df['run_number'] = i
            _, third_stop_iter_num = group_best_params_with_stop(
                third_group_df, stop_iter_num = stop_iter_num,
                left_iter_num=1, right_iter_num=third_group_df.shape[0]
            )

            third_stop_iter_num = third_stop_iter_num[i]
            third_group_df_cutted = third_group_df.loc[third_group_df.iter_num <= third_stop_iter_num]
            # concat and prepare:
            full_df = pd.concat([first_group_df_cutted, second_group_df_cutted, third_group_df_cutted])
            full_df['iter_num'] = range(1, full_df.shape[0]+1)
            print(DIR_TO_SAVE_RES + f"GRBO_groups_with_{stop_iter_num}_{i}_{dataset_name}.csv")
            full_df.to_csv(DIR_TO_SAVE_RES + f"GRBO_groups_with_{stop_iter_num}_{i}_{dataset_name}.csv", index=False)
            
        except:
            print(f"oh no: run_num {i}, stop criterion {stop_iter_num}")
            print("Second group stop num:", second_stop_iter_num)
            print("Iter limit:", 729 - (first_group_df_cutted.shape[0] + second_stop_iter_num))
#         finally:
#             dataset_name ='house_prices'
            

        

../../other_data_experiments/physics/fourth experiment/prepared/GRBO_groups_with_10_1_physics.csv
../../other_data_experiments/physics/fourth experiment/prepared/GRBO_groups_with_10_2_physics.csv
../../other_data_experiments/physics/fourth experiment/prepared/GRBO_groups_with_10_3_physics.csv
../../other_data_experiments/physics/fourth experiment/prepared/GRBO_groups_with_10_4_physics.csv
../../other_data_experiments/physics/fourth experiment/prepared/GRBO_groups_with_10_5_physics.csv
../../other_data_experiments/physics/fourth experiment/prepared/GRBO_groups_with_10_6_physics.csv
../../other_data_experiments/physics/fourth experiment/prepared/GRBO_groups_with_10_7_physics.csv
../../other_data_experiments/physics/fourth experiment/prepared/GRBO_groups_with_10_8_physics.csv
../../other_data_experiments/physics/fourth experiment/prepared/GRBO_groups_with_10_9_physics.csv
../../other_data_experiments/physics/fourth experiment/prepared/GRBO_groups_with_10_10_physics.csv
